In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/lib/kaggle/gcp.py
/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/working/__notebook_source__.ipynb


In [4]:
df_train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [5]:
from html import unescape

df_train["text"] = df_train["text"].apply(unescape)

In [6]:
!pip install sparknlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.2/473.2 kB 14.2 MB/s eta 0:00:00


In [7]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=a5c4e1fe3b96c45f2ef7c6c94857064cd1745b3a7f11d6c0f7466d57820e73ec
  Stored in directory: /root/.cache/pip/wheels/5a/54/9b/a89cac960efb57c4c35d41cc7c9f7b80daa21108bc376339b7
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [8]:
import sparknlp
spark = sparknlp.start() 
# sparknlp.start(gpu=True) >> for training on GPU
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)
# >> Spark NLP version 2.4.5
# >> Apache Spark version: 2.4.4

:: loading settings :: url = jar:file:/opt/conda/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1b2674fe-5dd3-4bc4-9e2b-eb683a17b87f;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.3.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.16.0 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failureaccess;1.0.1 

23/04/09 13:35:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/09 13:35:27 ERROR Inbox: Ignoring error
java.lang.NullPointerException
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$register(BlockManagerMasterEndpoint.scala:582)
	at org.apache.spark.storage.BlockManagerMasterEndpoint$$anonfun$receiveAndReply$1.applyOrElse(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:103)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100)
	at org.apache.spark.rpc.netty.MessageLoop.org$apache$spark$rpc$netty$MessageLoop$$receiveLoop(MessageLoop.scala:75)
	at org.apache.spark.rpc.netty.MessageLoop$$anon$1.run(MessageLoop.scala:41)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.

In [9]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("/kaggle/input/nlp-getting-started/train.csv")

In [10]:
trainDataset = trainDataset.filter((trainDataset.target == 0) | (trainDataset.target == 1))
trainDataset.show()

+---+-------+--------+--------------------+------+
| id|keyword|location|                text|target|
+---+-------+--------+--------------------+------+
|  1|   null|    null|Our Deeds are the...|     1|
|  4|   null|    null|Forest fire near ...|     1|
|  5|   null|    null|All residents ask...|     1|
|  6|   null|    null|13,000 people rec...|     1|
|  7|   null|    null|Just got sent thi...|     1|
|  8|   null|    null|#RockyFire Update...|     1|
| 10|   null|    null|#flood #disaster ...|     1|
| 13|   null|    null|I'm on top of the...|     1|
| 14|   null|    null|There's an emerge...|     1|
| 15|   null|    null|I'm afraid that t...|     1|
| 16|   null|    null|Three people died...|     1|
| 17|   null|    null|Haha South Tampa ...|     1|
| 18|   null|    null|#raining #floodin...|     1|
| 19|   null|    null|#Flood in Bago My...|     1|
| 20|   null|    null|Damage to school ...|     1|
| 23|   null|    null|      What's up man?|     0|
| 24|   null|    null|       I 

In [11]:
# from pyspark.ml.feature import OneHotEncoder
# encoder = OneHotEncoder(inputCols=['keyword', "location"], outputCols=['onehot'])  # , sparse=False)

In [12]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence
# downloading pretrained embeddings
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embedding")\
 
# densed1 = Vectors.dense("sentence_embeddings_densed")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
  .setInputCols("sentence_embedding")\
  .setOutputCol("class")\
  .setLabelColumn("target")\
  .setMaxEpochs(100)\
  .setEnableOutputLogs(True)  # \
#   .setTestDataset(df_test)

use_clf_pipeline = Pipeline(
    stages = [
        document, 
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]Download done! Loading the resource.
[ — ]

[OK!]


In [13]:
pipeline_model = use_clf_pipeline.fit(trainDataset)

Training started - epochs: 100 - learning_rate: 0.005 - batch_size: 64 - training_examples: 7176 - classes: 2
Epoch 1/100 - 1.54s - loss: 55.11395 - acc: 0.8066406 - batches: 113
Epoch 2/100 - 1.06s - loss: 53.212406 - acc: 0.8345424 - batches: 113
Epoch 3/100 - 1.07s - loss: 54.08964 - acc: 0.84737724 - batches: 113
Epoch 4/100 - 1.05s - loss: 53.475826 - acc: 0.8556083 - batches: 113
Epoch 5/100 - 1.01s - loss: 52.420216 - acc: 0.86160713 - batches: 113
Epoch 6/100 - 1.01s - loss: 51.026882 - acc: 0.86648995 - batches: 113
Epoch 7/100 - 1.04s - loss: 49.64494 - acc: 0.87262833 - batches: 113
Epoch 8/100 - 1.01s - loss: 48.847702 - acc: 0.8765346 - batches: 113
Epoch 9/100 - 1.01s - loss: 48.61727 - acc: 0.88044083 - batches: 113
Epoch 10/100 - 1.11s - loss: 48.505363 - acc: 0.88211495 - batches: 113
Epoch 11/100 - 1.15s - loss: 48.458088 - acc: 0.8836495 - batches: 113
Epoch 12/100 - 1.25s - loss: 48.46796 - acc: 0.88532364 - batches: 113
Epoch 13/100 - 1.08s - loss: 48.46932 - acc: 

In [16]:
# df_test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [15]:
testDataset = spark.read \
      .option("header", True) \
      .option('multiLine', 'true') \
      .csv("/kaggle/input/nlp-getting-started/test.csv")

predictions = pipeline_model.transform(testDataset)
df_predictions = predictions.toPandas()

def get_pred(line):
    _dict_probas = line[0][4]
    if float(_dict_probas['1']) > float(_dict_probas['0']):
        return 1
    else:
        return 0
    
final_preds = df_predictions["class"].apply(get_pred)
df_submit = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
df_submit["target"] = final_preds
df_submit.to_csv("universal_sentence_encoder_using-spark_nlp-100epochs.csv")